<center><font size="6"><b>Anticipez les besoins en consommation de bâtiments</b></font></center>


![Seattle City](Seattle-city.jpeg)



# Exploration des données de consommation énergétique des bâtiments non résidentiels

# Préambule

Ce notebook a pour objectif d'explorer les données de consommation énergétique des bâtiments non résidentiels de la ville, ainsi que les données structurelles associées à ces bâtiments. Les relevés minutieux de 2016 effectués par les agents de la ville seront utilisés comme données de référence. L'objectif est de tenter de prédire les émissions de CO2 et la consommation totale d'énergie de bâtiments non destinés à l'habitation pour lesquels ces données n'ont pas encore été mesurées. Nous chercherons également à évaluer l'intérêt de l'"ENERGY STAR Score" pour la prédiction d'émissions. Ce notebook comprendra une analyse exploratoire des données, ainsi que des tests de différents modèles de prédiction afin de répondre au mieux à la problématique. Nous ferons également attention au traitement des différentes variables, à la fois pour trouver de nouvelles informations et pour optimiser les performances en appliquant des transformations simples aux variables. Enfin, nous mettrons en place une évaluation rigoureuse des performances de la régression et optimiserons les hyperparamètres et le choix d'algorithmes de ML à l'aide d'une validation croisée.


<a id='sec:introduction'></a>
## I. Introduction

Ce premier notebook se concentre sur une étape essentielle du projet, à savoir l'analyse exploratoire des données et le prétraitement des données. Avant de plonger dans les différentes méthodes de prédiction, il est primordial de comprendre en profondeur les données de consommation énergétique des bâtiments non résidentiels de la ville de Seattle. Dans ce notebook, nous effectuerons une analyse approfondie des caractéristiques des bâtiments, des variables structurales et des relevés de consommation réalisés en 2016. Nous examinerons également les différentes étapes de prétraitement des données, telles que le traitement des valeurs manquantes et des données aberrantes. Cette étape est cruciale pour garantir des résultats fiables dans les étapes ultérieures de modélisation et de prédiction. Passons maintenant à la table des matières pour explorer en détail le contenu de ce premier notebook.

### Table des Matières
* [I. Introduction](#sec:introduction)
* [II. Analyse exploratoire des données](#sec:analyse-exploratoire)
    * A. Analyse univariée
    * B. Analyse bivariée
* [III. Prétraitement des données](#sec:pretraitement-donnees)
    * A. Traitement des valeurs manquantes
    * B. Traitement des données aberrantes
* [IV. Sélection de variables](#sec:selection-variables)
    * A. Analyse de la corrélation entre les variables
    * B. Analyse de la variance des variables
    * C. Feature Enginneering 
* [V. Conclusion](#sec:conc)
    * Analyse des corrélations entre les variables après Feature Engineering


<a id='sec:analyse-exploratoire'></a>
# II. Analyse exploratoire des données


Dans cette section, nous allons effectuer une analyse exploratoire des données pour mieux comprendre les caractéristiques des bâtiments de Seattle et la consommation d'énergie. Nous allons examiner les données en utilisant des techniques d'analyse univariée et bivariée, ainsi que des visualisations de données pour mettre en évidence les relations entre les variables. L'objectif de cette section est de déterminer les caractéristiques les plus importantes des bâtiments de Seattle qui influencent la consommation d'énergie.

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import missingno as msno
import plotly.express as px
import plotly.figure_factory as ff
from plotly import __version__
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
import chart_studio.plotly as py
import cufflinks as cf 
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import importlib.metadata

# For Notebooks
init_notebook_mode(connected=True)
# For offline use
cf.go_offline()
%matplotlib inline

In [2]:
#Modification des affichages de colonnes, lignes et largeurs de colonnes pour avoir un maximum d'information
pd.set_option('display.max_columns', 200)
pd.set_option('display.max_rows', 1000)
pd.set_option('display.max_colwidth', None)

In [3]:
!python --version

# Version des librairies utilisées
print('\n'.join(f'{m.__name__} - {m.__version__}' 
                for m in globals().values() 
                if getattr(m, '__version__', None)))

print("matplotlib - ", importlib.metadata.version('matplotlib'))

Python 3.8.3
pandas - 2.0.2
numpy - 1.23.5
seaborn - 0.12.1
missingno - 0.4.2
cufflinks - 0.17.3
matplotlib -  3.6.2


In [4]:
df = pd.read_csv('2016_Building_Energy_Benchmarking.csv')

In [5]:
df.head(10)

,OSEBuildingID,DataYear,BuildingType,PrimaryPropertyType,PropertyName,Address,City,State,ZipCode,TaxParcelIdentificationNumber,CouncilDistrictCode,Neighborhood,Latitude,Longitude,YearBuilt,NumberofBuildings,NumberofFloors,PropertyGFATotal,PropertyGFAParking,PropertyGFABuilding(s),ListOfAllPropertyUseTypes,LargestPropertyUseType,LargestPropertyUseTypeGFA,SecondLargestPropertyUseType,SecondLargestPropertyUseTypeGFA,ThirdLargestPropertyUseType,ThirdLargestPropertyUseTypeGFA,YearsENERGYSTARCertified,ENERGYSTARScore,SiteEUI(kBtu/sf),SiteEUIWN(kBtu/sf),SourceEUI(kBtu/sf),SourceEUIWN(kBtu/sf),SiteEnergyUse(kBtu),SiteEnergyUseWN(kBtu),SteamUse(kBtu),Electricity(kWh),Electricity(kBtu),NaturalGas(therms),NaturalGas(kBtu),DefaultData,Comments,ComplianceStatus,Outlier,TotalGHGEmissions,GHGEmissionsIntensity
0,1,2016,NonResidential,Hotel,Mayflower park hotel,405 Olive way,Seattle,WA,98101.0,0659000030,7,DOWNTOWN,47.61220,-122.33799,1927,1.0,12,88434,0,88434,Hotel,Hotel,88434.0,NaN,NaN,NaN,NaN,NaN,60.0,81.699997,84.300003,182.500000,189.000000,7226362.5,7456910.0,2.003882e+06,1.156514e+06,3946027.0,12764.529300,1276453.0,False,NaN,Compliant,NaN,249.98,2.83
1,2,2016,NonResidential,Hotel,Paramount Hotel,724 Pine street,Seattle,WA,98101.0,0659000220,7,DOWNTOWN,47.61317,-122.33393,1996,1.0,11,103566,15064,88502,"Hotel, Parking, Restaurant",Hotel,83880.0,Parking,15064.0,Restaurant,4622.0,NaN,61.0,94.800003,97.900002,176.100006,179.399994,8387933.0,8664479.0,0.000000e+00,9.504252e+05,3242851.0,51450.816410,5145082.0,False,NaN,Compliant,NaN,295.86,2.86
2,3,2016,NonResidential,Hotel,5673-The Westin Seattle,1900 5th Avenue,Seattle,WA,98101.0,0659000475,7,DOWNTOWN,47.61393,-122.33810,1969,1.0,41,956110,196718,759392,Hotel,Hotel,756493.0,NaN,NaN,NaN,NaN,NaN,43.0,96.000000,97.699997,241.899994,244.100006,72587024.0,73937112.0,2.156655e+07,1.451544e+07,49526664.0,14938.000000,1493800.0,False,NaN,Compliant,NaN,2089.28,2.19
3,5,2016,NonResidential,Hotel,HOTEL MAX,620 STEWART ST,Seattle,WA,98101.0,0659000640,7,DOWNTOWN,47.61412,-122.33664,1926,1.0,10,61320,0,61320,Hotel,Hotel,61320.0,NaN,NaN,NaN,NaN,NaN,56.0,110.800003,113.300003,216.199997,224.000000,6794584.0,6946800.5,2.214446e+06,8.115253e+05,2768924.0,18112.130860,1811213.0,False,NaN,Compliant,NaN,286.43,4.67
4,8,2016,NonResidential,Hotel,WARWICK SEATTLE HOTEL (ID8),401 LENORA ST,Seattle,WA,98121.0,0659000970,7,DOWNTOWN,47.61375,-122.34047,1980,1.0,18,175580,62000,113580,"Hotel, Parking, Swimming Pool",Hotel,123445.0,Parking,68009.0,Swimming Pool,0.0,NaN,75.0,114.800003,118.699997,211.399994,215.600006,14172606.0,14656503.0,0.000000e+00,1.573449e+06,5368607.0,88039.984380,8803998.0,False,NaN,Compliant,NaN,505.01,2.88
5,9,2016,Nonresidential COS,Other,West Precinct,810 Virginia St,Seattle,WA,98101.0,0660000560,7,DOWNTOWN,47.61623,-122.33657,1999,1.0,2,97288,37198,60090,Police Station,Police Station,88830.0,NaN,NaN,NaN,NaN,NaN,NaN,136.100006,141.600006,316.299988,320.500000,12086616.0,12581712.0,0.000000e+00,2.160444e+06,7371434.0,47151.816410,4715182.0,False,NaN,Compliant,NaN,301.81,3.10
6,10,2016,NonResidential,Hotel,Camlin,1619 9th Avenue,Seattle,WA,98101.0,0660000825,7,DOWNTOWN,47.61390,-122.33283,1926,1.0,11,83008,0,83008,Hotel,Hotel,81352.0,NaN,NaN,NaN,NaN,NaN,27.0,70.800003,74.500000,146.600006,154.699997,5758795.0,6062767.5,0.000000e+00,8.239199e+05,2811215.0,29475.800780,2947580.0,False,NaN,Compliant,NaN,176.14,2.12
7,11,2016,NonResidential,Other,Paramount Theatre,911 Pine St,Seattle,WA,98101.0,0660000955,7,DOWNTOWN,47.61327,-122.33136,1926,1.0,8,102761,0,102761,Other - Entertainment/Public Assembly,Other - Entertainment/Public Assembly,102761.0,NaN,NaN,NaN,NaN,NaN,NaN,61.299999,68.800003,141.699997,152.300003,6298131.5,7067881.5,2.276286e+06,1.065843e+06,3636655.0,3851.890137,385189.0,False,NaN,Compliant,NaN,221.51,2.16
8,12,2016,NonResidential,Hotel,311wh-Pioneer Square,612 2nd Ave,Seattle,WA,98104.0,0939000080,7,DOWNTOWN,47.60294,-122.33263,1904,1.0,15,163984,0,163984,Hotel,Hotel,163984.0,Na

In [6]:
df.columns.tolist()

['OSEBuildingID',
 'DataYear',
 'BuildingType',
 'PrimaryPropertyType',
 'PropertyName',
 'Address',
 'City',
 'State',
 'ZipCode',
 'TaxParcelIdentificationNumber',
 'CouncilDistrictCode',
 'Neighborhood',
 'Latitude',
 'Longitude',
 'YearBuilt',
 'NumberofBuildings',
 'NumberofFloors',
 'PropertyGFATotal',
 'PropertyGFAParking',
 'PropertyGFABuilding(s)',
 'ListOfAllPropertyUseTypes',
 'LargestPropertyUseType',
 'LargestPropertyUseTypeGFA',
 'SecondLargestPropertyUseType',
 'SecondLargestPropertyUseTypeGFA',
 'ThirdLargestPropertyUseType',
 'ThirdLargestPropertyUseTypeGFA',
 'YearsENERGYSTARCertified',
 'ENERGYSTARScore',
 'SiteEUI(kBtu/sf)',
 'SiteEUIWN(kBtu/sf)',
 'SourceEUI(kBtu/sf)',
 'SourceEUIWN(kBtu/sf)',
 'SiteEnergyUse(kBtu)',
 'SiteEnergyUseWN(kBtu)',
 'SteamUse(kBtu)',
 'Electricity(kWh)',
 'Electricity(kBtu)',
 'NaturalGas(therms)',
 'NaturalGas(kBtu)',
 'DefaultData',
 'Comments',
 'ComplianceStatus',
 'Outlier',
 'TotalGHGEmissions',
 'GHGEmissionsIntensity']

__*OSE Building ID:*__ identifiant unique de chaque bâtiment 

__*Data Year :*__ année des données

__*Building Type :*__ type de bâtiment (commercial, multifamilial, etc.)
 
__*Primary Property Type :*__ type de propriété principale (bureau, hôtel, etc.)

__*Property Name :*__ nom du bâtiment

__*Address :*__ adresse du bâtiment

__*City :*__ ville où se trouve le bâtiment

__*State :*__ état où se trouve le bâtiment

__*ZipCode :*__ code postal du bâtiment

__*Tax Parcel Identification Number :*__ numéro d'identification fiscale du bâtiment

__*Council District Code :*__ code du district municipal où se trouve le bâtiment

__*Neighborhood :*__ quartier où se trouve le bâtiment

__*Latitude :*__ latitude géographique du bâtiment

__*Longitude :*__ longitude géographique du bâtiment

__*YearBuilt :*__ année de construction du bâtiment

__*Number of Buildings :*__ nombre de bâtiments sur le site

__*Number of Floors :*__ nombre d'étages dans le bâtiment

__*Property GFA Total :*__ superficie totale du bâtiment (en pieds carrés)

__*Property GFA Parking :*__ superficie de stationnement du bâtiment (en pieds carrés)

__*Property GFA Building(s) :*__ superficie du ou des bâtiments (en pieds carrés)

__*List Of All Property Use Types :*__ liste de tous les types d'utilisation de la propriété

__*Largest Property Use Type :*__ le plus grand type d'utilisation de la propriété (par superficie)

__*Largest Property Use Type GFA :*__ superficie de l'utilisation principale de la propriété (en pieds carrés)

__*Second Largest Property Use Type :*__ deuxième plus grand type d'utilisation de la propriété (par superficie)

__*Second Largest Property Use Type GFA :*__ superficie de la deuxième utilisation la plus importante de la propriété (en pieds carrés)

__*Third Largest Property Use Type :*__ troisième plus grand type d'utilisation de la propriété (par superficie)

__*Third Largest Property Use Type GFA :*__ superficie de la troisième utilisation la plus importante de la propriété (en pieds carrés)

__*Years ENERGY STAR Certified :*__ nombre d'années où le bâtiment a été certifié ENERGY STAR

__*ENERGY STAR Score :*__ score ENERGY STAR pour le bâtiment

__*Site EUI (kBtu/sf) :*__ indicateur de performance énergétique du site (énergie utilisée par pied carré de surface)

__*Site EUIWN (kBtu/sf) :*__ indicateur de performance énergétique du site ajusté en fonction des conditions météorologiques (énergie utilisée par pied carré de surface)

__*Source EUI (kBtu/sf) :*__ indicateur de performance énergétique de la source (énergie utilisée par pied carré de surface)

__*Source EUIWN (kBtu/sf) :*__ indicateur de performance énergétique de la source ajusté en fonction des conditions météorologiques (énergie utilisée par pied carré de surface)

__*Site Energy Use (kBtu) :*__ consommation d'énergie du site (en milliers de British Thermal Units)

__*Site Energy Use WN (kBtu) :*__ consommation d'énergie du site ajustée en fonction des conditions météorologiques (en milliers de British Thermal Units)

__*Electricity (kWh) :*__ consommation d'électricité (en kilowattheures)

__*Electricity (kBtu) :*__ consommation d'électricité (en milliers de British Thermal Units)

__*Natural Gas (therms) :*__ consommation de gaz naturel en thermies

__*Natural Gas (kBtu) :*__ consommation de gaz naturel (en milliers de British Thermal Units)

__*Default Data :*__ indication si les données sont les données par défaut (True/False)

__*Comments :*__ commentaires sur les données

__*Compliance Status :*__ état de conformité de l'immeuble avec la réglementation de benchmarking de l'énergie de Seattle

__*Outlier :*__ indication si les données sont des valeurs aberrantes (True/False)

__*Total GHG Emissions :*__ total des émissions de gaz à effet de serre en kilogrammes d'équivalent dioxyde de carbone (CO2e)

__*GHG Emissions Intensity :*__ intensité des émissions de gaz à effet de serre en kilogrammes d'équivalent dioxyde de carbone (CO2e) par pied carré (unité de mesure de superficie)

In [8]:
df.shape

(3376, 46)

In [7]:
# Compter le nombre de valeurs manquantes et calculer le pourcentage de NaN dans le dataframe
missing_values = pd.DataFrame(df.isna().sum(), columns = ['Total de valeurs manquantes'])
missing_values['Pourcentage de valeurs manquantes'] = round((missing_values['Total de valeurs manquantes'] / len(df)) * 100, 2)

# Sélectionner uniquement les colonnes avec des valeurs manquantes
missing_values = missing_values.query("`Total de valeurs manquantes` > 0")

# Trier les colonnes par ordre décroissant de nombre de valeurs manquantes
missing_values = missing_values.sort_values(by = ['Total de valeurs manquantes'], ascending=False)
display(missing_values)

,Total de valeurs manquantes,Pourcentage de valeurs manquantes
Comments,3376,100.00
Outlier,3344,99.05
YearsENERGYSTARCertified,3257,96.48
ThirdLargestPropertyUseTypeGFA,2780,82.35
ThirdLargestPropertyUseType,2780,82.35
SecondLargestPropertyUseType,1697,50.27
SecondLargestPropertyUseTypeGFA,1697,50.27
ENERGYSTARScore,843,24.97
LargestPropertyUseType,20,0.59
LargestPropertyUseTypeGFA,20,0.59


## A. Analyse univariée

In [ ]:
df.hist(figsize=(45,45))


Dans le graphique ci-dessus, vous pouvez voir les histogrammes de toutes les variables numériques. Certaines variables sont réparties dans une plage de valeurs spécifique, tandis que d'autres ont une distribution en forme de cloche avec des valeurs qui augmentent de manière logarithmique. Les variables Zipcode et Coincil District Code prennent des valeurs catégorielles spécifiques. Il est important d'examiner attentivement ces variables et de les préparer pour des méthodes de machine learning à l'aide de techniques de feature engineering appropriées.


In [ ]:
df['Neighborhood'].value_counts()

In [ ]:
#Convertir toutes les chaînes de caractères en majuscules en utilisant la méthode 
df['Neighborhood'] = df['Neighborhood'].str.upper()
df['Neighborhood'] = df['Neighborhood'].str.replace('DELRIDGE NEIGHBORHOODS', 'DELRIDGE')
df['Neighborhood'].value_counts(normalize=True)

 Il semble que la majorité des données proviennent des quartiers centraux de la ville. Les cinq premiers quartiers représentent 60% des données totales. Cela peut être pris en compte lors de l'analyse et de la modélisation des données, car cela peut entraîner un biais dans les prévisions pour les quartiers moins représentés dans les données.

In [ ]:
color_var = pd.factorize(df['Neighborhood'])[0]

fig = px.scatter_mapbox(df, lat="Latitude", lon="Longitude", hover_name="Neighborhood",
                        color=color_var, zoom=9, height=300)
fig.update_layout(mapbox_style="open-street-map")
fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
fig.show()


Comme vous pouvez le voir dans la figure ci-dessus, vous pouvez visualiser la répartition des noms de quartiers sur la carte. De cette manière, vous pouvez voir où se situent les différents quartiers sur la carte. Il est probable que la consommation d'énergie dans les bâtiments puisse varier dans différents quartiers, et ces différences peuvent être utilisées dans l'ingénierie des fonctionnalités.

In [ ]:
plt.figure(figsize=(15, 10))
sns.countplot(x=df['BuildingType'])
plt.xticks(rotation=90, fontsize=16)
plt.yticks(fontsize=16)
plt.xlabel('Building Type', fontsize=18)
plt.ylabel('Count', fontsize=18)
plt.title('Distribution of Building Types', fontsize=22)
plt.show()


In [ ]:
df['BuildingType'].value_counts()

Lorsqu'on examine la colonne "BuildingType", on peut voir qu'il y a plusieurs types de bâtiments différents, notamment "NonResidential", "Multifamily", "SPS-District K-12", "Nonresidential COS" et "Campus". Étant donné que ce projet se concentre uniquement sur les bâtiments "NonResidential", les types "Multifamily" peuvent être exclus de l'ensemble de données. De plus, puisqu'il n'y a qu'un seul exemple de données appartenant à "Nonresidential WA", ces données peuvent être supprimées car elles ne pourront pas être utilisées dans la phase de modélisation.

In [ ]:
df[[ 'BuildingType',
 'PrimaryPropertyType',
 'SecondLargestPropertyUseType',
 'ThirdLargestPropertyUseType',
 'LargestPropertyUseType',
 'ListOfAllPropertyUseTypes'
    ]].head(50)

Effectivement, chaque type de bâtiment peut comporter différents types d'espaces ou de locaux tels que des restaurants, des bureaux, des entrepôts, etc. Par conséquent, la colonne "ListOfAllPropertyUseTypes" fournit des informations plus détaillées sur les différents types de bâtiments. Je vais utiliser la colonne de données "PrimaryPropertyType" pour la visualisation car ces données sont plus propres et généralisées que les autres.

In [ ]:
df['PrimaryPropertyType'].value_counts()

In [ ]:
property_type_colors = {
    'Low-Rise Multifamily': 'purple',
    'Mid-Rise Multifamily': 'violet',
    'Small- and Mid-Sized Office': 'blue',
    'Other': 'black',
    'Warehouse': 'gray',
    'Large Office': 'navy',
    'K-12 School': 'green',
    'Mixed Use Property': 'orange',
    'High-Rise Multifamily': 'deep pink',
    'Retail Store': 'red',
    'Hotel': 'brown',
    'Worship Facility': 'olive',
    'Distribution Center': 'dark gray',
    'Senior Care Community': 'light pink',
    'Supermarket / Grocery Store': 'maroon',
    'Medical Office': 'teal',
    'Self-Storage Facility': 'dark orange',
    'University': 'forest green',
    'Residence Hall': 'magenta',
    'Refrigerated Warehouse': 'light gray',
    'Restaurant': 'crimson',
    'Hospital': 'dark red',
    'Laboratory': 'lime',
    'Office': 'sky blue'
}


# Create the scattermapbox plot using the property_type_colors dictionary
fig = px.scatter_mapbox(df, lat="Latitude", lon="Longitude", color="PrimaryPropertyType",
                        color_discrete_map=property_type_colors, zoom=10, height=600)

# Update the layout with the mapbox style
fig.update_layout(mapbox_style="open-street-map")

# Show the plot
fig.show()


Dans la carte ci-dessus, en examinant les emplacements des données, on peut voir que les riches familles habitent principalement dans le centre-ville et que la plupart des données sont des logements multifamiliaux. La plupart des données se concentrent dans les quartiers du centre-ville. Les centres de distribution de Seattle se trouvent plutôt au sud et près de l'aéroport, tandis que les écoles sont réparties dans toute la ville.

In [ ]:
# Créer une liste contenant les noms des colonnes à convertir
cols_to_convert = ['PropertyGFATotal', 'PropertyGFAParking', 'PropertyGFABuilding(s)', 'LargestPropertyUseTypeGFA', 'SecondLargestPropertyUseTypeGFA', 'ThirdLargestPropertyUseTypeGFA']

# Définir la constante de conversion (de pieds carrés à mètres carrés)
SQ_FT_TO_SQ_M = 0.092903

# Parcourir la liste des colonnes à convertir
for col in cols_to_convert:
    # Convertir les valeurs de la colonne en mètres carrés en multipliant par la constante de conversion
    df[col] = df[col] * SQ_FT_TO_SQ_M

# rename the columns to indicate that they are now in square meters
df = df.rename(columns={
    'PropertyGFATotal': 'PropertyGFATotal_m2',
    'PropertyGFAParking': 'PropertyGFAParking_m2',
    'PropertyGFABuilding(s)': 'PropertyGFABuilding(s)_m2',
    'LargestPropertyUseTypeGFA': 'LargestPropertyUseTypeGFA_m2',
    'SecondLargestPropertyUseTypeGFA': 'SecondLargestPropertyUseTypeGFA_m2',
    'ThirdLargestPropertyUseTypeGFA': 'ThirdLargestPropertyUseTypeGFA_m2'
})


In [ ]:
# calculate 90th percentile of PropertyGFATotal_m2
threshold = df['PropertyGFATotal_m2'].quantile(0.90)
# create a subset of the dataframe with values below the threshold
df_no_outliers = df[df['PropertyGFATotal_m2'] < threshold]

# create a histogram of PropertyGFATotal_m2 without outliers
df_no_outliers['PropertyGFATotal_m2'].iplot(kind='hist', bins=40)


In [ ]:
df.loc[df['PropertyGFATotal_m2'].idxmax()]


In [ ]:
df.loc[df['SiteEnergyUse(kBtu)'].idxmax()]



D'après les données, le complexe universitaire est l'endroit avec la consommation d'énergie la plus élevée et la superficie de bâtiment la plus élevée. Il est composé de 111 bâtiments.

In [ ]:
zero_floors = df[df['NumberofFloors'] == 0]
for index, row in zero_floors.iterrows():
    print("Building Type: ", row['BuildingType'], ':', row['ListOfAllPropertyUseTypes'])


Comme il est intéressant de constater, ce complexe de logements est enregistré avec un nombre d'étages de 0. En examinant de plus près les bâtiments avec ce nombre d'étages, on constate que la plupart d'entre eux sont des dortoirs d'étudiants et certains ont été mal enregistrés. Il est donc nécessaire de nettoyer ces données en conséquence.

In [ ]:
df["YearBuilt"].iplot(kind='hist')


Pour convertir la colonne "YearBuilt" en âge du bâtiment en considérant l'année actuelle comme 2023, nous pouvons utiliser la formule suivante :

In [ ]:
import datetime

current_year = datetime.datetime.now().year
df["BuildingAge"] = current_year - df["YearBuilt"]

In [ ]:
df['BuildingAge'].iplot(kind='box')

La moyenne d'âge des bâtiments est de 46 ans et la plupart des valeurs se situent entre 26 et 75 ans. Le bâtiment le plus ancien a 123 ans

## B. Analyse bivariée

In [ ]:
plt.figure(figsize=(15,10))
sns.boxplot(x='BuildingType', y='NumberofFloors', data=df)
plt.xticks(rotation=60, fontsize=16)
plt.yticks(fontsize=16)
plt.xlabel('Building Type', fontsize=18)
plt.ylabel('Number of Floors', fontsize=18)
plt.show()



Comme on peut le voir sur ce boxplot, les gratte-ciels à plusieurs étages sont généralement de type de bâtiment non résidentiel. Les résidences multifamiliales et les campus contiennent des bâtiments moins élevés.

In [ ]:
df['NumberofBuildings'].value_counts()

Dans la colonne "Nombre de bâtiments", il y a une valeur de 0 qui n'est pas logique. Par conséquent, j'ai corrigé cela en la remplaçant par 1.

In [ ]:
df['NumberofBuildings'] = df['NumberofBuildings'].replace(0, 1)


In [ ]:
df = df[df['NumberofBuildings'] != 111]

J'ai également supprimé la ligne de données qui comprend 111 bâtiments. Comme nous l'avons constaté, il s'agit d'une zone du campus et cela ne correspond pas au projet. Cela pourrait entraîner une erreur lors de l'utilisation de méthodes ML.

In [ ]:
fig = px.box(df, x='BuildingType', y='NumberofBuildings')
fig.update_layout(
    xaxis_title='Building Type',
    yaxis_title='Number of Buildings',
    font=dict(size=18),
    width=1000,
    height=600
)
fig.show()



Le nombre de bâtiments sur le campus est plus élevé que celui des autres types de bâtiments.

In [ ]:
plt.figure(figsize=(50,50))
sns.set(font_scale=2.5)
numeric_columns = df.select_dtypes(include=['int32', 'float64']).columns

plt.title('Matrice de corrélation de Pearson entre les différentes features', fontsize=50)

corr = df[numeric_columns].corr()

mask = np.zeros_like(corr, dtype=bool)
mask[np.triu_indices_from(mask)] = True 

ax = sns.heatmap(corr, mask=mask, vmin=-1, cmap='coolwarm', annot=True, annot_kws={'size':35})
ax.tick_params(axis='both', labelsize=50)
plt.xticks(rotation=90, fontsize=50,fontweight='bold')
plt.yticks(rotation=0, fontsize=50,fontweight='bold')

plt.show()

Le score Energystars est négativement corrélé avec les valeurs de consommation des bâtiments. D'autre part, la quantité annuelle d'énergie consommée par la propriété est fortement corrélée avec la consommation d'électricité et la valeur totale d'émissions de gaz à effet de serre (GES). Sa corrélation avec l'utilisation de gaz naturel et de vapeur n'est également pas négligeable. L'autre corrélation élevée est liée à la surface de plancher brute et à la consommation d'énergie.

<a id='sec:pretraitement-donnees'></a>
# III. Prétraitement des données

### A. Traitement des valeurs manquantes

In [ ]:
msno.bar(df)

La plupart des données sont complètes. Les données manquantes sont pour la plupart inutiles à traiter. La superficie brute des bâtiments (GFA) n'est pas définie pour les bâtiments de type de propriété secondaire. Je vais laisser la colonne "EnergyStarScore" intacte, car j'ai besoin des valeurs de l'ENERGY STAR Score pour pouvoir prédire certaines valeurs énergétiques à l'avenir. Maintenant, je vais commencer le nettoyage des données.

In [ ]:
df['ZipCode'].nunique()

Dans ce code, nous utilisons la bibliothèque Geopy pour géocoder les adresses. La fonction geocode_address prend une adresse en entrée et renvoie les coordonnées géographiques correspondantes. Nous itérons ensuite sur chaque ligne du DataFrame pour prédire le code postal en utilisant la fonction geocode_address

In [ ]:
import geopy
from geopy.geocoders import Nominatim
import re

def geocode_address(address):
    geolocator = Nominatim(user_agent="my-app") 
    location = geolocator.geocode(address)
    return location

# Parcourir les lignes du DataFrame et prédire le code postal pour chaque adresse
for index, row in df.iterrows():
    address = row['Address']
    zip_code = row['ZipCode']

    if pd.isnull(zip_code):  # Vérifier si le code postal est NaN
        location = geocode_address(address)

        if location is not None:
            try:
                zip_code = location.raw['address']['postcode']
            except KeyError:
                display_name = location.raw['display_name']
                zip_code_match = re.search(r'Washington, (\d{5})', display_name)
                if zip_code_match:
                    zip_code = zip_code_match.group(1)
                else:
                    zip_code = None

            df.loc[index, 'ZipCode'] = zip_code
        else:
            df.loc[index, 'ZipCode'] = None


In [ ]:
df['ZipCode'].value_counts()


In [ ]:
df[df['ZipCode'].isna()==True]['ZipCode']


In [ ]:
df['ZipCode'] = pd.to_numeric(df['ZipCode'], errors='coerce')
df.loc[3372,'ZipCode'] = 98104.0
df.loc[3361,'ZipCode'] = 98144.0

In [ ]:
df.drop('Comments', inplace=True,axis=1)

### B. Traitement des données aberrantes

In [ ]:
df['Outlier'].value_counts()

Dans la colonne "Outlier", j'ai choisi de les supprimer en raison de la faible quantité de données, mais on pourrait également les utiliser en utilisant OneHotEncoder.

In [ ]:
df = df.drop(df[df['Outlier'].isin(['Low outlier', 'High outlier'])].index)

In [ ]:
df.drop('Outlier', inplace=True,axis=1)

In [ ]:
df['BuildingType'].value_counts()

In [ ]:
df = df.drop(df[df['BuildingType'].isin(['Multifamily LR (1-4)', 'Multifamily MR (5-9)', 'Multifamily HR (10+)', 'Nonresidential WA'])].index)


Dans ce projet, nous nous concentrons uniquement sur les appartements non-résidentiels. C'est pourquoi nous supprimons les lignes de données qui ne correspondent pas à cette sélection.

Étant donné qu'il n'y a qu'une seule donnée dans la catégorie des bâtiments Nonresidential WA  nous l'avons supprimée. La généralisation de cette donnée n'est pas très importante pour les algorithmes d'apprentissage automatique (ML).

In [ ]:
df['ComplianceStatus'].value_counts()

J'ai également supprimé certaines lignes qui contenaient des données incomplètes ou manquantes dans la colonne "ComplianceStatus".

In [ ]:
df = df[df['ComplianceStatus']=='Compliant']

In [ ]:
df['PrimaryPropertyType'].value_counts()

In [ ]:
df[df['PrimaryPropertyType']=='Low-Rise Multifamily']

Même si j'ai supprimé les données contenant la catégorie 'Multifamily Housing', certaines données sont encore présentes dans la variable 'PrimaryPropertyType'

In [ ]:
# Remplacer les valeurs manquantes par une chaîne de caractères vide
df["LargestPropertyUseType"].fillna("", inplace=True)

# Filtrer les lignes qui contiennent "Multifamily Housing"
filtre = df["LargestPropertyUseType"].str.contains("Multifamily Housing")
resultats = df[filtre]

# Imprimer les résultats
display(resultats)



Il reste encore quelques catégories de logements multifamiliaux sous la colonne "largestPropertyUseType". Dans le cadre de ce projet, j'ai décidé de négliger ces colonnes afin de me concentrer uniquement sur les bâtiments non résidentiels.

In [ ]:
df = df[~filtre]

In [ ]:
# Définition des colonnes d'énergie secondaires
sub_energy_cols = ['SteamUse(kBtu)', 'Electricity(kBtu)', 'NaturalGas(kBtu)']

# Calcul de la somme des énergies secondaires pour chaque ligne
df['SiteEnergyUse(kBtu)_total'] = df[sub_energy_cols].sum(axis='columns')

# Définition des limites inférieure et supérieure de la plage [90%-110%]
lower_bound = df['SiteEnergyUse(kBtu)'] * 0.9
upper_bound = df['SiteEnergyUse(kBtu)'] * 1.1

# Vérification si la somme des énergies n'est pas comprise dans la plage [90%-110%] de 'SiteEnergyUse(kBtu)'
m_en_out = ~df['SiteEnergyUse(kBtu)_total'].between(lower_bound, upper_bound)

# Affichage du nombre de lignes où l'énergie totale n'est pas dans la plage [90%-110%] par rapport à 'SiteEnergyUse(kBtu)'
print("Nombre de lignes où l'énergie totale n'est pas dans la plage [90%-110%] de 'SiteEnergyUse(kBtu)': ",
      df[m_en_out].shape[0], '/', df.shape[0])


In [ ]:
fig = px.scatter(df, x='SiteEnergyUse(kBtu)_total', y='SiteEnergyUse(kBtu)', color='LargestPropertyUseType')
fig.show()

Il y a 15 données pour lesquelles les valeurs d'énergie utilisée ne correspondent pas à la somme des valeurs d'utilisation d'électricité, de gaz naturel et de vapeur. Cela signifie que ces bâtiments pourraient utiliser d'autres sources d'énergie qui ne sont pas répertoriées dans l'ensemble de données.

In [ ]:
sub_GFA_cols1 = ['PropertyGFAParking_m2', 'PropertyGFABuilding(s)_m2']

# Calculer la surface brute totale (GFA) pour chaque ligne
df['PropertyGFATotal_total'] = df[sub_GFA_cols1].sum(axis='columns')

# Calculer l'écart en pourcentage par rapport à 'PropertyGFATotal' d'origine
df['Écart_GFA_en_pourcentage'] = (df['PropertyGFATotal_total'] / df['PropertyGFATotal_m2']) * 100

# Filtrer les lignes où la GFA ne se situe pas dans la plage [99%-101%] par rapport à 'PropertyGFATotal'
m_GFA_out = ~df['Écart_GFA_en_pourcentage'].between(99, 101)

# Compter le nombre de lignes où la GFA est en dehors de la plage [99%-101%]
num_GFA_out = df[m_GFA_out].shape[0]

# Afficher le résultat
print(f"Nombre de lignes où la surface brute totale (GFA) n'est pas dans la plage [99%-101%] de 'PropertyGFATotal': {num_GFA_out} / {df.shape[0]}")

# Supprimer les colonnes intermédiaires
df = df.drop(columns=['PropertyGFATotal_total', 'Écart_GFA_en_pourcentage'])


La somme de la surface de stationnement et de la surface du bâtiment est corrélée avec le nombre total de mètres carrés.

### Z-score

In [ ]:
nom_colonne1 = 'TotalGHGEmissions'
nom_colonne2 = 'SiteEnergyUse(kBtu)'

# Calculer les scores Z pour les colonnes
scores_z1 = np.abs((df[nom_colonne1] - df[nom_colonne1].mean()) / df[nom_colonne1].std())
scores_z2 = np.abs((df[nom_colonne2] - df[nom_colonne2].mean()) / df[nom_colonne2].std())

# Trouver les valeurs aberrantes en utilisant un seuil
seuil = 3
valeurs_aberrantes1 = df[scores_z1 > seuil][nom_colonne1]
valeurs_aberrantes2 = df[scores_z2 > seuil][nom_colonne2]

# Créer un tableau pour afficher les valeurs aberrantes
tableau_valeurs_aberrantes = pd.DataFrame({
    nom_colonne1: valeurs_aberrantes1,
    nom_colonne2: valeurs_aberrantes2
})

# Afficher le tableau
print("Tableau des valeurs aberrantes:")
print(tableau_valeurs_aberrantes)


Nous avons obtenu des valeurs aberrantes pour les colonnes TotalGHGEmissions et SiteEnergyUse(kBtu). Beaucoup d'entre elles partagent également le même indice. Par conséquent, nous avons décidé de supprimer les lignes de données qui pourraient affecter les méthodes de ML.

In [ ]:
# Supprimer les données aberrantes du DataFrame
df = df.drop(tableau_valeurs_aberrantes.index)


<a id='sec:selection-variables'></a>
# IV. Sélection de variables

### A. Analyse de la corrélation entre les variables

J'ai supprimé les colonnes qui ne contiennent qu'une seule valeur unique, puis j'ai supprimé certaines colonnes qui sont sans pertinence dans le cadre de ce projet.

In [ ]:
for col in df.columns:
    if df[col].nunique() == 1:
        df.drop(col, axis=1, inplace=True)


In [ ]:
df['PropertyName'].value_counts()

In [ ]:
df.drop('PropertyName',axis=1, inplace=True)

In [ ]:
df.drop('TaxParcelIdentificationNumber',axis=1, inplace=True)

La plupart des données de consommation énergétique élevée proviennent de l'hôtel qui possède la plus grande superficie brute.

In [ ]:
fig = px.scatter(df, x='PropertyGFATotal_m2', y='SiteEnergyUse(kBtu)', color='LargestPropertyUseType')
fig.show()


Les laboratoires, les supermarchés et les hôpitaux utilisent généralement plus d'énergie, mais la catégorie "Other" regroupe des valeurs énergétiques très diverses.

In [ ]:
fig = px.box(df, x='PrimaryPropertyType', y='SiteEUI(kBtu/sf)',
             width=800, height=600, color_discrete_sequence=px.colors.qualitative.Pastel)

fig.show()


### B. Analyse de la variance des variables

J'ai réalisé une visualisation de la distribution du score ENERGY STAR en utilisant une estimation de la densité par noyau (KDE). Cette visualisation nous permet d'observer la répartition des scores ENERGY STAR dans notre jeu de données. La méthode KDE nous donne une estimation de la densité de probabilité, ce qui nous permet de mieux comprendre la variabilité des scores. Cette analyse de la variance des variables nous aide à identifier les variations et les tendances dans les scores ENERGY STAR,

In [ ]:
df['ENERGYSTARScore'].iplot(kind='hist')

In [ ]:
# drop rows with NaN values in 'ENERGYSTARScore' column
score_data = df['ENERGYSTARScore'].dropna()

# create the histogram with KDE
fig = ff.create_distplot([score_data], ['Energy Star Score'], bin_size=1, curve_type='kde')

# update layout and display the figure
fig.update_layout(title='Energy Star Score Distribution with KDE', xaxis_title='Score', yaxis_title='Density')
fig.show()


J'ai effectué plusieurs visualisations pour analyser la variance des variables . Tout d'abord, j'ai tracé un histogramme de la variable 'SiteEUI(kBtu/sf)' afin d'examiner la distribution de la consommation d'énergie par pied carré. Cela nous permet de mieux comprendre la répartition des valeurs et de détecter d'éventuelles tendances ou valeurs aberrantes.

In [ ]:
df['SiteEUI(kBtu/sf)'].iplot(kind='hist')

Ensuite, j'ai regroupé les années de construction des bâtiments en catégories de 10 ans (par exemple, 1900-1909, 1910-1919, etc.) et calculé la moyenne du score ENERGY STAR pour chaque groupe d'années. J'ai représenté ces moyennes sous forme de diagramme à barres afin de visualiser les variations du score d'efficacité énergétique au fil des années. Cela nous permet d'analyser l'évolution de l'efficacité énergétique des bâtiments en fonction de leur année de construction.

In [ ]:
# create a new column for year groups
df['YearGroup'] = pd.cut(df['YearBuilt'], bins=range(df['YearBuilt'].min(), df['YearBuilt'].max()+11, 10), labels=range(df['YearBuilt'].min(), df['YearBuilt'].max(), 10))

# calculate the average energy score for each year group
df_grouped = df.groupby('YearGroup')['ENERGYSTARScore'].mean().reset_index()

# create the bar plot
fig = px.bar(df_grouped, x='YearGroup', y='ENERGYSTARScore', labels={'YearGroup': 'Year Group', 'ENERGYSTARScore': 'Average Energy Score'})
fig.show()


Enfin, j'ai créé des diagrammes de dispersion pour examiner les relations entre différentes variables liées à la consommation et à l'utilisation d'énergie. J'ai tracé des diagrammes de dispersion pour les paires de variables suivantes : 'SourceEUI' vs 'SourceEUIWN', 'SiteEUI' vs 'SiteEUIWN', et 'SiteEnergyUse' vs 'SiteEnergyUseWN'. Ces diagrammes de dispersion nous permettent d'analyser les corrélations et les tendances entre ces variables, ce qui est important pour comprendre les facteurs influençant la consommation d'énergie des bâtiments.

In [ ]:
# Create a scatter plot for SourceEUI and SourceEUIWN
fig1 = px.scatter(df, x='SourceEUI(kBtu/sf)', y='SourceEUIWN(kBtu/sf)')

# Create a scatter plot for SiteEUI and SiteEUIWN
fig2 = px.scatter(df, x='SiteEUI(kBtu/sf)', y='SiteEUIWN(kBtu/sf)')

# Create a scatter plot for SiteEnergyUse and SiteEnergyUseWN
fig3 = px.scatter(df, x='SiteEnergyUse(kBtu)', y='SiteEnergyUseWN(kBtu)')

# Show the plots
fig1.show()
fig2.show()
fig3.show()


Les valeurs d'énergie ajustées pour représenter ce que la propriété aurait consommé lors de conditions météorologiques moyennes sur 30 ans montrent une tendance similaire aux valeurs non ajustées. Cependant, certaines valeurs ajustées sont manquantes et donnent des valeurs nulles. Le reste des valeurs ajustées est similaire aux valeurs non ajustées.

In [ ]:
color_var = pd.factorize(df['SiteEUI(kBtu/sf)'])[0]

fig = px.scatter_mapbox(df, lat="Latitude", lon="Longitude", hover_name="SiteEUI(kBtu/sf)",
                        color=color_var, zoom=9, height=300)
fig.update_layout(mapbox_style="open-street-map")
fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
fig.show()

In [ ]:
plt.figure(figsize=(12, 6))
ax = sns.boxplot(data=df, x='Neighborhood', y='SiteEnergyUse(kBtu)')

# Increase font size of axis labels
ax.set_xlabel('Quartier', fontsize=20)
ax.set_ylabel('Consommation d\'énergie (kBtu)', fontsize=20)

plt.xticks(fontsize=12, rotation=45)  # Increase font size of x-axis labels and rotate them for better visibility

plt.show()


La consommation d'énergie n'est pas directement proportionnelle à la latitude et à la longitude. Cependant, dans le centre-ville, le taux de consommation d'énergie est plus élevé que dans le reste de la ville.

### C. Feature Engineering

Dans cette section du notebook, nous aborderons l'ingénierie des caractéristiques (feature engineering). Nous explorerons différentes techniques pour créer de nouvelles variables à partir des données existantes, telles que la transformation des variables, la création de variables d'interaction et la sélection de variables pertinentes. L'objectif est d'améliorer la performance de nos modèles de prédiction en fournissant des informations supplémentaires et en réduisant le bruit dans les données.

### Energy Type

In [ ]:
df['SteamRatio'] = df['SteamUse(kBtu)'] / df['SiteEnergyUse(kBtu)']
df['ElectricityRatio'] = df['Electricity(kBtu)'] / df['SiteEnergyUse(kBtu)']
df['NaturalGasRatio'] = df['NaturalGas(kBtu)'] / df['SiteEnergyUse(kBtu)']

 Cette étape d'ingénierie des caractéristiques vise à capturer les contributions relatives de différentes sources d'énergie dans la consommation d'énergie globale des bâtiments.

In [ ]:
df[['SteamUse(kBtu)', 'Electricity(kBtu)', 'NaturalGas(kBtu)']].idxmax(axis=1).value_counts()

Parmi les trois types d'énergie, l'électricité est le plus utilisé, suivi du gaz naturel. Le type d'énergie le moins utilisé est la vapeur. 

In [ ]:
# Création de nouvelles colonnes avec des valeurs initiales définies à 0
df['most_steam'] = 0
df['most_electricity'] = 0
df['most_gas'] = 0

# Recherche de la colonne avec la valeur la plus élevée dans chaque ligne et configuration de la nouvelle colonne correspondante à 1
max_column = df[['SteamUse(kBtu)', 'Electricity(kBtu)', 'NaturalGas(kBtu)']].idxmax(axis=1)
df.loc[max_column == 'SteamUse(kBtu)', 'most_steam'] = 1
df.loc[max_column == 'Electricity(kBtu)', 'most_electricity'] = 1
df.loc[max_column == 'NaturalGas(kBtu)', 'most_gas'] = 1

Dans ce code, nous créons de nouvelles colonnes avec des valeurs initiales définies à 0. Ensuite, nous recherchons la colonne avec la valeur la plus élevée dans chaque ligne et nous configurons la nouvelle colonne correspondante à 1. Ainsi, les colonnes "plus_de_vapeur", "plus_d_electricite" et "plus_de_gaz" auront une valeur de 1 si la vapeur, l'électricité ou le gaz ont la plus grande consommation respective dans chaque bâtiment.

In [ ]:
# Prendre le logarithme des valeurs d'énergie
df['log_energie'] = np.log(df['SiteEnergyUse(kBtu)'])

# Regrouper les données par quartier et calculer la moyenne de SiteEnergyUse(kBtu) pour chaque groupe
moyenne_par_quartier = df.groupby('Neighborhood')['log_energie'].mean()

# Créer un DataFrame trié
ordre_quartier = moyenne_par_quartier.sort_values(ascending=False).index

# Créer le diagramme en boîte à moustaches avec Plotly
fig = px.violin(df, x='Neighborhood', y='log_energie', category_orders={'Neighborhood': ordre_quartier})
fig.show()


Le logarithme est souvent utilisé pour transformer les variables cibles dans le contexte de l'apprentissage automatique. L'utilisation du logarithme permet de réduire l'échelle des valeurs et de rendre la distribution des données plus symétrique

In [ ]:
# Prendre le logarithme des valeurs d'émissions
df['log_emission'] = np.log(df['TotalGHGEmissions'].replace(0, np.nan))

# Regrouper les données par quartier et calculer la moyenne de log_emission pour chaque groupe
moyenne_par_quartier = df.groupby('Neighborhood')['log_emission'].mean()

# Créer un DataFrame trié
ordre_quartier = moyenne_par_quartier.sort_values(ascending=False).index

# Créer le diagramme en boîte avec Plotly
fig = px.box(df, x='Neighborhood', y='log_emission', category_orders={'Neighborhood': ordre_quartier})
fig.show()

In [ ]:
sum(df['log_emission'].isna())

In [ ]:
df.dropna(subset=['log_emission'], inplace=True)


In [ ]:
df.shape

De plus, la transformation logarithmique peut aider à atténuer l'effet des valeurs aberrantes et à rendre les relations entre les variables plus linéaires, ce qui peut améliorer les performances des modèles d'apprentissage automatique

### Mean GFA per floor

In [ ]:
df["PropertyGFABuilding(s)_m2"].describe()

Nous créons une nouvelle caractéristique appelée 'MeanGFAperFloor', qui représente la surface brute moyenne par étage d'un bâtiment.

In [ ]:
df['MeanGFAperFloor'] = df['PropertyGFABuilding(s)_m2']/(df['NumberofFloors']+1)


Nous ajoutons 1 au dénominateur pour éviter une division par zéro au cas où un bâtiment n'aurait aucun étage.

### Parking GFA ratio

In [ ]:
df['PropertyGFAParking_m2'].describe()

nous créons une nouvelle caractéristique appelée 'ParkingGFARatio', qui représente le ratio de la surface dédiée au stationnement par rapport à la surface totale du bâtiment.

In [ ]:
df['ParkingGFARatio'] = df['PropertyGFAParking_m2']/df['PropertyGFATotal_m2']


In [ ]:
df['ParkingGFARatio'].describe()

## Largest Property Use Type

In [ ]:
df['LargestPropertyUseType'].value_counts()

In [ ]:
df = df[df['LargestPropertyUseType'] != '']

In [ ]:
df['LargestPropertyUseType'].value_counts()

Nous avons créé un mapping des types de bâtiments aux noms de catégories. Chaque type de bâtiment est associé à une catégorie correspondante.

In [ ]:
# Mapping of building types to category names
building_category_mapping = {
    'Office': 'Commercial Facilities',
    'Non-Refrigerated Warehouse': 'Industrial Facilities',
    'Retail Store': 'Commercial Facilities',
    'Other': 'Miscellaneous Facilities',
    'Hotel': 'Miscellaneous Facilities',
    'Worship Facility': 'Miscellaneous Facilities',
    'Distribution Center': 'Industrial Facilities',
    'K-12 School': 'Educational Facilities',
    'Medical Office': 'Healthcare Facilities',
    'Supermarket/Grocery Store': 'Commercial Facilities',
    'Other - Recreation': 'Entertainment and Recreation Facilities',
    'Parking': 'Miscellaneous Facilities',
    'Self-Storage Facility': 'Miscellaneous Facilities',
    'Residence Hall/Dormitory': 'Residential Facilities',
    'Other - Entertainment/Public Assembly': 'Entertainment and Recreation Facilities',
    'College/University': 'Educational Facilities',
    'Senior Care Community': 'Healthcare Facilities',
    'Laboratory': 'Educational Facilities',
    'Restaurant': 'Entertainment and Recreation Facilities',
    'Refrigerated Warehouse': 'Miscellaneous Facilities',
    'Social/Meeting Hall': 'Miscellaneous Facilities',
    'Hospital (General Medical & Surgical)': 'Healthcare Facilities',
    'Manufacturing/Industrial Plant': 'Industrial Facilities',
    'Strip Mall': 'Commercial Facilities',
    'Repair Services (Vehicle, Shoe, Locksmith, etc)': 'Industrial Facilities',
    'Fitness Center/Health Club/Gym': 'Entertainment and Recreation Facilities',
    'Museum': 'Entertainment and Recreation Facilities',
    'Other - Lodging/Residential': 'Residential Facilities',
    'Automobile Dealership': 'Miscellaneous Facilities',
    'Other - Services': 'Miscellaneous Facilities',
    'Other - Mall': 'Miscellaneous Facilities',
    'Other/Specialty Hospital': 'Healthcare Facilities',
    'Financial Office': 'Commercial Facilities',
    'Library': 'Miscellaneous Facilities',
    'Bank Branch': 'Commercial Facilities',
    'Urgent Care/Clinic/Other Outpatient': 'Healthcare Facilities',
    'Other - Education': 'Educational Facilities',
    'Performing Arts': 'Entertainment and Recreation Facilities',
    'Prison/Incarceration': 'Miscellaneous Facilities',
    'Data Center': 'Industrial Facilities',
    'Lifestyle Center': 'Miscellaneous Facilities',
    'Other - Public Services': 'Miscellaneous Facilities',
    'Other - Restaurant/Bar': 'Miscellaneous Facilities',
    'Adult Education': 'Educational Facilities',
    'Other - Utility': 'Miscellaneous Facilities',
    'Pre-school/Daycare': 'Residential Facilities',
    'Fire Station': 'Miscellaneous Facilities',
    'Wholesale Club/Supercenter': 'Miscellaneous Facilities',
    'Residential Care Facility': 'Healthcare Facilities',
    'Police Station': 'Miscellaneous Facilities',
    'Food Service': 'Miscellaneous Facilities',
    'Movie Theater': 'Entertainment and Recreation Facilities',
    'Personal Services (Health/Beauty, Dry Cleaning, etc)': 'Miscellaneous Facilities',
    'Courthouse': 'Miscellaneous Facilities'
}

# Create the category_building column based on the mapping
df['category_building'] = df['LargestPropertyUseType'].map(building_category_mapping).fillna('Other')

# Display the updated DataFrame
print(df[['LargestPropertyUseType', 'category_building']])


In [ ]:
df['category_building'].value_counts()

Pour chaque catégorie, nous avons créé une nouvelle colonne avec le nom de la catégorie suivi de '_ratio'. Cette colonne représente le ratio de la surface 'LargestPropertyUseTypeGFA_m2' par rapport à la surface totale 'PropertyGFATotal_m2' pour les bâtiments de cette catégorie. Les valeurs manquantes sont remplies avec 0.

In [ ]:
# Convert 'category_building' column back to string data type
df['category_building'] = df['category_building'].astype(str)

# Iterate over each unique value in 'category_building'
for category in df['category_building'].unique():
    # Create a new column with the unique value as the column name
    category_str = str(category)
    df[category_str + '_ratio'] = df.loc[df['category_building'] == category, 'LargestPropertyUseTypeGFA_m2'] / df.loc[df['category_building'] == category, 'PropertyGFATotal_m2']
    df[category_str + '_ratio'] = df[category_str + '_ratio'].fillna(0)


### Ratio Second and third largest 

Ces étapes permettent de calculer les ratios de surfaces pour les deuxièmes et troisièmes plus grandes utilisations de propriétés, en les rapportant à la surface totale du bâtiment. 

In [ ]:
df['SecondLargestPropertyUseTypeGFA_m2'].fillna(0, inplace=True)
df['ThirdLargestPropertyUseTypeGFA_m2'].fillna(0, inplace=True)
df['SLP_Ratio'] = df['SecondLargestPropertyUseTypeGFA_m2'] / df['PropertyGFATotal_m2']
df['TLP_Ratio'] = df['ThirdLargestPropertyUseTypeGFA_m2'] / df['PropertyGFATotal_m2']


### Years EnergySTAR Certified

In [ ]:
df['YearsENERGYSTARCertified'].value_counts()


Nous avons défini une fonction appelée 'split_dates' pour diviser les dates en années individuelles. Cette fonction gère les cas où les dates sont fournies sous forme d'une chaîne séparée par des virgules ou sous forme d'une chaîne continue sans virgules.

In [ ]:
def split_dates(dates):
    dates_str = str(dates).replace('nan', '')
    if ',' in dates_str:
        ls_date = [s.strip() for s in str(dates_str).split(',')]
    else:
        if len(dates_str)%4==0:
            ls_date = [dates_str[4*(i):4*(i+1)] for i in range(int(len(dates_str)/4))]
        else:
            print("ERROR: ", dates_str)
    return tuple(ls_date)


df['YearsENERGYSTARCertified'] = df['YearsENERGYSTARCertified']\
                                             .apply(lambda x: split_dates(x))

In [ ]:
df['YearsENERGYSTARCertified'].value_counts()

Nous avons créé une nouvelle colonne appelée 'NumYearsENERGYSTARCertified' qui représente le nombre d'années pendant lesquelles une propriété a été certifiée ENERGY STAR. Cela a été réalisé en appliquant la fonction 'len' à chaque valeur de tuple.

In [ ]:
df['NumYearsENERGYSTARCertified'] = df['YearsENERGYSTARCertified'].apply(lambda x: len(x))


In [ ]:
import datetime

# Get the current year
current_year = datetime.datetime.now().year

# Function to extract the first year from the tuple or return None if it's empty
def get_first_year(years):
    if isinstance(years, tuple) and len(years) > 0:
        return years[-1]
    return None


# Extract the first year from the tuple
df['FirstCertifiedYear'] = df['YearsENERGYSTARCertified'].apply(get_first_year)



Nous avons défini une fonction appelée 'get_first_year' pour extraire la première année (la plus récente) du tuple ou retourner None si le tuple est vide. Nous avons appliqué la fonction 'get_first_year' à la colonne 'YearsENERGYSTARCertified' et assigné l'année extraite à la colonne 'FirstCertifiedYear'.

In [ ]:

# Convert 'LastCertifiedYear' column to numeric type
df['FirstCertifiedYear'] = pd.to_numeric(df['FirstCertifiedYear'], errors='coerce')

# Perform the calculation (2023 - df['LastCertifiedYear'])
df['YearsSinceCertified'] = 2023 - df['FirstCertifiedYear']

# Replace NaN values in 'YearsSinceCertified' column with 0
df['YearsSinceCertified'] = df['YearsSinceCertified'].fillna(0)


Nous avons calculé le nombre d'années depuis la première certification en soustrayant 'FirstCertifiedYear' de l'année actuelle (2023) et avons stocké le résultat dans la colonne 'YearsSinceCertified'.

<a id='sec:conc'></a>
# V. Conclusion

### Analyse des corrélations entre les variables après Feature Engineering

Nous avons réalisé une ingénierie des caractéristiques sur notre ensemble de données, ce qui nous a permis de créer de nouvelles variables basées sur des calculs et des transformations des variables existantes

In [ ]:
fig = make_subplots(rows=1, cols=2)

# Add the first subplot
fig.add_trace(
    go.Scatter(x=df['ENERGYSTARScore'], y=df['YearsSinceCertified'], mode='markers'),
    row=1, col=1
)
fig.update_xaxes(title_text='ENERGYSTARScore', row=1, col=1)
fig.update_yaxes(title_text='YearsSinceCertified', row=1, col=1)

# Add the second subplot
fig.add_trace(
    go.Scatter(x=df['ENERGYSTARScore'], y=df['NumYearsENERGYSTARCertified'], mode='markers'),
    row=1, col=2
)
fig.update_xaxes(title_text='ENERGYSTARScore', row=1, col=2)
fig.update_yaxes(title_text='NumYearsENERGYSTARCertified', row=1, col=2)

# Update layout and display the subplot
fig.update_layout(title='ENERGYSTARScore Subplots')
fig.show()

Si un bâtiment est certifié plusieurs fois, cela lui donne un score plus élevé dans le classement ENERGY STAR.

Nous avons effectué un test de corrélation pour évaluer les relations entre les nouvelles variables.

In [ ]:
# Sélectionner les colonnes pour la heatmap
col_num = ['ENERGYSTARScore', 'SiteEnergyUse(kBtu)', 'SiteEUI(kBtu/sf)', 'SourceEUI(kBtu/sf)', 'BuildingAge',
           'MeanGFAperFloor', 'ParkingGFARatio', 'SteamRatio', 'NumberofBuildings', 'NumberofFloors',
           'PropertyGFATotal_m2', 'ElectricityRatio', 'NaturalGasRatio', 'log_energie', 'log_emission',
           'TotalGHGEmissions', 'GHGEmissionsIntensity', 'YearsSinceCertified', 'NumYearsENERGYSTARCertified']

# Créer une matrice de corrélation en utilisant la corrélation de Pearson
correlation_matrix = df[col_num].corr(method='pearson')

# Générer un masque pour le triangle supérieur
mask = np.triu(np.ones_like(correlation_matrix, dtype=bool))

# Définir la taille de la figure
plt.figure(figsize=(30, 30))

# Générer la heatmap avec les valeurs masquées
heatmap = sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm', mask=mask)

# Augmenter la taille de la police des index sur l'axe x et y
heatmap.set_xticklabels(heatmap.get_xticklabels(), fontsize=25)
heatmap.set_yticklabels(heatmap.get_yticklabels(), fontsize=25)

# Augmenter la taille de la police des nombres à l'intérieur de la heatmap
for text in heatmap.texts:
    text.set_fontsize(20)
    text.set_weight('bold')

# Définir le titre
plt.title('Heatmap de corrélation de Pearson des colonnes liées à l\'énergie', fontsize=25)

# Afficher le graphique
plt.show()


Ce code sélectionne uniquement les colonnes numériques du DataFrame, calcule la matrice de corrélation entre ces variables, filtre la matrice de corrélation en ne conservant que les valeurs absolues supérieures à 0.5 (et différentes de 1), puis supprime les colonnes où toutes les corrélations sont inférieures à 0.5. Enfin, il affiche la matrice de corrélation filtrée.

In [ ]:
# Sélectionner uniquement les colonnes numériques
numerical_df = df.select_dtypes(include=['float64', 'int64'])

# Calculer la matrice de corrélation
correlation_matrix = numerical_df.corr()

# Filtrer la matrice de corrélation en fonction des valeurs absolues supérieures à 0.5
filtered_corr_matrix = correlation_matrix[(abs(correlation_matrix) > 0.5) & (correlation_matrix != 1)]

# Supprimer les colonnes où toutes les corrélations sont inférieures à 0.5
filtered_corr_matrix = filtered_corr_matrix.dropna(how='all', axis=0).dropna(how='all', axis=1)

# Afficher la matrice de corrélation filtrée
filtered_corr_matrix


In [ ]:
# Créer une heatmap triangulaire
sns.set(style="white")
mask = np.triu(np.ones_like(filtered_corr_matrix, dtype=bool))
fig, ax = plt.subplots(figsize=(80, 50))
heatmap = sns.heatmap(filtered_corr_matrix, mask=mask, cmap='coolwarm', annot=True, fmt=".2f", linewidths=2, ax=ax)

# Augmenter la taille de la police de tous les textes
heatmap.set_xticklabels(heatmap.get_xticklabels(), fontsize=70)
heatmap.set_yticklabels(heatmap.get_yticklabels(), fontsize=70)
heatmap.set_title("Heatmap de la matrice de corrélation filtrée", fontsize=150)

# Augmenter la taille de la police des nombres à l'intérieur de la heatmap
for text in heatmap.texts:
    text.set_fontsize(50)
    text.set_weight('bold')

plt.show()

Dans ce notebook, j'ai principalement réalisé des études exploratoires pour découvrir les relations entre les différentes colonnes et extraire les données les plus pertinentes à utiliser dans la section de prédiction. J'ai sélectionné la section des bâtiments non résidentiels, nettoyé les données, identifié et supprimé les valeurs aberrantes. Enfin, j'ai travaillé sur la partie de l'ingénierie des caractéristiques afin d'obtenir des données utiles que les méthodes d'apprentissage automatique peuvent bien comprendre.

In [ ]:
# Enregistrer le DataFrame nettoyé dans un fichier CSV

df.to_csv('data_clean.csv', index=False)
